<a href="https://colab.research.google.com/github/Mar-vm/VitalIA/blob/backend/Vitalia3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()  # Sube kaggle.json

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"


Saving kaggle.json to kaggle.json


In [13]:
!chmod 600 /content/kaggle.json


In [14]:
# DESCARGAR DATASET DE KAGGLE
from zipfile import ZipFile

!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# Descomprimir el archivo zip descargado
with ZipFile('skin-cancer-mnist-ham10000.zip', 'r') as zip_ref:
    zip_ref.extractall('skin_cancer_data')  # Puedes cambiar el directorio de destino si lo prefieres

print("Dataset descargado y descomprimido.")


Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
skin-cancer-mnist-ham10000.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset descargado y descomprimido.


In [15]:
# EXPLORAR DATASET
import os

# Verificar el contenido del directorio 'skin_cancer_data'
dataset_dir = '/content/skin_cancer_data'
print(os.listdir(dataset_dir))


['HAM10000_metadata.csv', 'hmnist_28_28_RGB.csv', 'ham10000_images_part_2', 'hmnist_28_28_L.csv', 'hmnist_8_8_RGB.csv', 'HAM10000_images_part_2', 'ham10000_images_part_1', 'hmnist_8_8_L.csv', 'HAM10000_images_part_1']


In [4]:
# LIBRERIAS NECESARIAS PARA PROCESAMIENTO DE IMAGENES
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os


In [16]:
# TRANSFORMACION DE IMAGENES
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Definir las transformaciones a aplicar a las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar para el modelo
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalización
])

# Cargar las imágenes usando ImageFolder
dataset = datasets.ImageFolder(dataset_dir, transform=transform)

# Crear un DataLoader para cargar las imágenes en lotes
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Ver el tamaño de la dataset y las clases
print(f"Total de imágenes: {len(dataset)}")
print(f"Clases: {dataset.classes}")



Total de imágenes: 20030
Clases: ['HAM10000_images_part_1', 'HAM10000_images_part_2', 'ham10000_images_part_1', 'ham10000_images_part_2']


In [6]:
# CREAR DATASET PERSONALIZADO
class SkinDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform
        self.image_names = os.listdir(image_dir)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_name)
        label = self.labels[self.image_names[idx]]  # Asume que tienes un diccionario de etiquetas

        if self.transform:
            image = self.transform(image)

        return image, label


In [17]:
# CARGAR Y ENTRENAR MODELO EFFICIENT NET

# Cargar EfficientNet B0 preentrenado
model = models.efficientnet_b0(pretrained=True)

# Cambiar la última capa para que coincida con el número de clases en tu dataset
num_ftrs = model.classifier[1].in_features
num_classes = len(dataset.classes)  # El número de clases es el tamaño de las carpetas

# Reemplazar la capa final del modelo
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Definir el criterio de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# ENTRENAMIENTO
num_epochs = 10  # Puedes ajustar el número de épocas según necesites

# Ciclo de entrenamiento
for epoch in range(num_epochs):
    model.train()  # Poner el modelo en modo de entrenamiento
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Borrar gradientes

        # Paso hacia adelante
        outputs = model(inputs)
        loss = criterion(outputs, labels)  # Calcular la pérdida

        # Paso hacia atrás y optimización
        loss.backward()
        optimizer.step()

        # Estadísticas
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Mostrar la pérdida y precisión por época
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%")


In [ ]:
torch.save(model.state_dict(), '/content/model_skin_cancer.pth')


In [9]:
# FUNCION Y PERDIDA
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
